## Assignment 2
## Weight: 20%
## Unit - COMP329: Artificial Intelligence
## Date of Release: 18 October, 2018
## Date of Submission: 07 November, 2018 [11: 55 pm]


### Write your Details below:

# The goal of this assignment is to appreciate the power of Evolutionary Algorithms (e.g., Genetic Algorithm, or GA in short) for solving real-world problems such as the Travelling Salesman Problem (TSP) and simple adversarial games such as Tic-Tac-Toe.

## Travelling Salesman Problem:
### Acknowledgment: 
#### The following is largely based on Peter Norvig's note on TSP: (https://github.com/norvig/pytudes/blob/master/ipynb/TSP.ipynb)
#### Given a set of cities and the distance between each pair of cities, what is the shortest possible tour that visits each city exactly once, and returns to the starting city?

<img src="tsp.png">

#### Let's us define the problem more precisely
- ***Given a set of cities***
<br>A Python `set` could represent a set of cities. An individual city might be just an integer index, or it might be (x, y) coordinates.
- ... ***and the distance between each pair of cities***: 
<br>We could use either a function, `distance(A, B),` or a table, `distance[A, B]`.
- ... ***what is the shortest possible tour***
<br>A tour is a sequential order in which to visit the cities; a function `shortest_tour(tours)` should find the one that minimizes `tour_length(tour)`, which is the sum of the distances between adjacent cities in the tour. 
- ... ***that visits each city once and returns to the starting city***
<br>Make sure a tour doesn't re-visit a city (except returning to the start).

### The *vocabulary* of the problem:

- **City**: For the purpose of this exercise, a city is "atomic" in the sense that we don't have to know anything about the components or attributes of a city, just how far it is from other cities.
- **Cities**: We will need to represent a set of cities; Python's `set` datatype might be appropriate for that.
- **Distance**: We will need the distance between two cities.  If `A` and `B` are cities. This could be done with a function, `distance(A, B)`, or with a dict, `distance[A][B]` or `distance[A, B]`, or with an array if `A` and `B` are integer indexes.  The resulting distance will be a real number (which Python calls a `float`).
- **Tour**: A tour is an ordered list of cities; Python's `list` or `tuple` datatypes would work.
- **Total distance**: The sum of the distances of adjacent cities in the tour. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import random, operator, time, itertools, math
import numpy

%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
#plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

### First algorithm: find the tour with shortest total distance from all possible tours

> *Generate all the possible tours of the cities, and choose the shortest one (the tour with the minimum total distance).*

### Representing Tours

- A tour starts in one city, and then visits each of the other cities in order, before finally retirning to the start. 
- A natural representation of the set of available cities is a Python `set`, and a natural representation of a tour is a sequence that is a *permutation* of the set. 
- The tuple `(1, 2, 3)`, for example, represents a tour that starts in city 1, moves to 2, then 3, and then returns to 1 to finish the tour.

In [2]:
alltours = itertools.permutations
cities = {1, 2, 3}
list(alltours(cities))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

### Representing Cities and Distance

Now for the notion of *distance*.  We define `total_distance(tour)` as the sum of the distances between consecutive cities in the tour; that part is shown below and is easy (with one Python-specific trick: when `i` is 0, then `distance(tour[0], tour[-1])` gives us the wrap-around distance between the first and last cities, because `tour[-1]` is the last element of `tour`). 


In [3]:
def total_distance(tour):
    "The total distance between each pair of consecutive cities in the tour."
    return sum(distance(tour[i], tour[i-1]) 
               for i in range(len(tour)))

In [4]:
def exact_TSP(cities):
    "Generate all possible tours of the cities and choose the shortest one."
    return shortest(alltours(cities))

def shortest(tours): 
    "Return the tour with the minimum total distance."
    return min(tours, key=total_distance)

### Representing distance between cities

In [5]:
City = complex # Constructor for new cities, e.g. City(300, 400)

In [6]:
def distance(A, B): 
    "The Euclidean distance between two cities."
    return abs(A - B)

In [7]:
# An example to show the distance between city A and city B
A = City(300, 0)
B = City(0, 400)
distance(A, B)

500.0

In [8]:
# function to generate n cities randomly using random number generator
def generate_cities(n):
    "Make a set of n cities, each with random coordinates."
    return set(City(random.randrange(10, 890), 
                    random.randrange(10, 590)) 
               for c in range(n))

In [9]:
# Generating cities
cities8, cities10, cities100, cities1000 = generate_cities(8), generate_cities(10), generate_cities(100), generate_cities(1000)

In [10]:
# Getting coordinates for 10 cities
cities10

{(110+396j),
 (118+323j),
 (187+355j),
 (341+352j),
 (385+100j),
 (409+61j),
 (571+514j),
 (600+329j),
 (745+184j),
 (832+578j)}

### functions to plot the tour

In [11]:
def plot_tour(tour, alpha=1, color=None):
    # Plot the tour as blue lines between blue circles, and the starting city as a red square.
    plotline(list(tour) + [tour[0]], alpha=alpha, color=color)
    plotline([tour[0]], style='gD', alpha=alpha, size=10)
    plt.show()
    
def plotline(points, style='bo-', alpha=1, size=7, color=None):
    "Plot a list of points (complex numbers) in the 2-D plane."
    X, Y = XY(points)
    
    if color:
        plt.plot(X, Y, style, alpha=alpha, markersize=size, color=color)
    else:
        plt.plot(X, Y, style, alpha=alpha, markersize=size)
    
def XY(points):
    "Given a list of points, return two lists: X coordinates, and Y coordinates."
    return [p.real for p in points], [p.imag for p in points]

In [ ]:
tour = exact_TSP(cities10)
plot_tour(tour)

### Removing redundant tours to increase efficiency
The permutation `(1, 2, 3)` represents the tour that goes from 1 to 2 to 3 and back to 1.  You may have noticed that there aren't really six different tours of three cities: the cities 1, 2, and 3 form a triangle;  any tour must connect the three points of the triangle; and there are really only two ways to do this: clockwise or counterclockwise.   In general, with $n$ cities, there are $n!$ (that is, $n$ factorial) permutations, but only  $(n-1)!$,  tours that are *distinct*: the tours `123`, `231`, and `312` are three ways of representing the *same* tour.

So we can make our `TSP` program $n$ times faster by never considering redundant tours. Arbitrarily, we will say that all tours must start with the "first" city in the set of cities. We don't have to change the definition of `TSP`&mdash;just by making `alltours` return only nonredundant tours, the whole program gets faster.


In [ ]:
def all_non_redundant_tours(cities):
    "Return a list of tours, each a permutation of cities, but each one starting with the same city."
    start = first(cities)
    return [[start] + list(tour)
            for tour in itertools.permutations(cities - {start})]

def first(collection):
    "Start iterating over collection, and return the first element."
    for x in collection: return x

def exact_non_redundant_TSP(cieaties):
    "Generate all possible tours of the cities and choose the shortest one."
    return shortest(all_non_redundant_tours(cities))

In [ ]:
all_non_redundant_tours({1, 2, 3})

### finding time based on First Approach: Exhaustive Search

In [ ]:
%timeit exact_TSP(cities8)

In [ ]:
%timeit exact_non_redundant_TSP(cities8)

## Second approach: Approximate (Heuristic) algorithms

### Greedy approach (Nearest Neighbor approach)
> *Start at any city; at each step extend the tour by moving from the previous city to its nearest neighbor that has not yet been visited.*

This is called a *greedy algorithm*, because it greedily takes what looks best in the short term (the nearest neighbor) even when that won't always be the best in the long term. 

In [ ]:
def greedy_TSP(cities):
    "At each step, visit the nearest neighbor that is still unvisited."
    start = first(cities)
    tour = [start]
    unvisited = cities - {start}
    while unvisited:
        C = nearest_neighbor(tour[-1], unvisited)
        tour.append(C)
        unvisited.remove(C)
    return tour

In [ ]:
def nearest_neighbor(A, cities):
    "Find the city in cities that is nearest to city A."
    return min(cities, key=lambda x: distance(x, A))

In [ ]:
cities = generate_cities(10)

In [ ]:
%timeit exact_non_redundant_TSP(cities)

In [ ]:
plot_tour(exact_non_redundant_TSP(cities))

In [ ]:
%timeit greedy_TSP(cities)

In [ ]:
plot_tour(greedy_TSP(cities))

Comparing the time taken to get results for 10 cities, it is clear that greedy approach is efficient compared to the exhaustive search.

A [greedy algorithm](http://en.wikipedia.org/wiki/Greedy_algorithm) is an algorithm that follows the problem solving heuristic of making the locally optimal choice at each stage with the hope of finding a global optimum. In many problems, a greedy strategy does not in general produce an optimal solution, but nonetheless a greedy heuristic may yield locally optimal solutions that approximate a global optimal solution in a reasonable time.

For many problmes greedy algorithms fail to produce the optimal solution, and may even produce the *unique worst possible solution*.

### A thought on computational complexity

<img src='http://imgs.xkcd.com/comics/travelling_salesman_problem.png' align='center' width='65%'/>


# Biologically inspired metaheuristic: Genetic Algorithm (GA)

- We have already studied GA in our lectures and did practice in our labs for one-max problem
- They are an option in which we dedicate a little more computational effort in order to produce better solutions than `greedy_TSP()`.

> We will be using the [DEAP](https://github.com/DEAP/deap) library to code this tackle this problem using a genetic algorithm. We have used DEAP in practical classes in this unit.

[<img src='https://raw.githubusercontent.com/DEAP/deap/master/doc/_static/deap_long.png' width='29%' align='center'/>](https://github.com/DEAP/deap)

In [ ]:
from deap import algorithms, base, creator, tools

In [ ]:
num_cities = 30
cities = generate_cities(num_cities)

In [ ]:
toolbox = base.Toolbox()

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
toolbox.register("indices", numpy.random.permutation, len(cities))
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)
toolbox.register("population", tools.initRepeat, list, 
                 toolbox.individual)

In [ ]:
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)

In [ ]:
def create_tour(individual):
    return [list(cities)[e] for e in individual]

In [ ]:
def evaluation(individual):
    '''Evaluates an individual by converting it into 
    a list of cities and passing that list to total_distance'''
    return total_distance(create_tour(individual))

In [ ]:
toolbox.register("evaluate", evaluation)

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
pop = toolbox.population(n=100)

In [ ]:
%%time 
result, log = algorithms.eaSimple(pop, toolbox,
                             cxpb=0.8, mutpb=0.2,
                             ngen=400, verbose=False)

Let's check the efficiency of GA

In [ ]:
best_individual = tools.selBest(result, k=1)[0]
print('Fitness of the best individual: ', evaluation(best_individual))

In [ ]:
plot_tour(create_tour(best_individual))

It is interesting to assess how the fitness of the population changes as the evolution process proceeds. 

We can prepare a `deap.tools.Statistics` instance to specify what data to collect. 

In [ ]:
fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
fit_stats.register('mean', numpy.mean)
fit_stats.register('min', numpy.min)

In [ ]:
result, log = algorithms.eaSimple(toolbox.population(n=100), toolbox,
                                  cxpb=0.5, mutpb=0.2,
                                  ngen=400, verbose=False,
                                  stats=fit_stats)

#### Plotting mean and minimum fitness values as evolution proceeds

In [ ]:
plt.figure(figsize=(11, 4))
plots = plt.plot(log.select('min'),'c-', log.select('mean'), 'b-')
plt.legend(plots, ('Minimum fitness', 'Mean fitness'), frameon=True)
plt.ylabel('Fitness'); plt.xlabel('Iterations');

### Q1. Compare the efficiency (time taken) of Exhaustive search, greedy search and Genetic Algorithm for 30 cities based on the configuration of your machine. If necessary, use estimates, but detail the reason for, and the basis of, the estimates used.  [ 1 Marks]

In [ ]:
# Write your (expected) output and analysis here
num_cities = 30
cities = generate_cities(num_cities)
#%timeit exact_TSP(cities)
#%timeit greedy_TSP(cities)
#%time result, log = algorithms.eaSimple(pop, toolbox,cxpb=0.8, mutpb=0.2,nngen=400, verbose=False)
#%timeit tools.selBest(result, k=1)[0]

### Q2. Analyse the effect of population size for the above experiment. Try population size having value [25, 50, 75, 100, 125, 150, 175, 200] [ 1.5 Marks]

In [ ]:
# Write your code here
num_cities = 30
cities = generate_cities(num_cities)
x = [25, 50, 75, 100, 125, 150, 175, 200]
for each in x:
    pop = toolbox.population(n=each)
    %time result, log = algorithms.eaSimple(pop, toolbox,cxpb=0.8, mutpb=0.2,ngen=400, verbose=False)
    best_individual = tools.selBest(result, k=1)[0]
    create_tour(best_individual)

### Q3. What is the influence  of mutation probability and cross-over probability over the performance of  GA. [ 1.5 Marks]

In [ ]:
# Write your code here

for each in x:
    pop = toolbox.population(n=each)
    %time result, log = algorithms.eaSimple(pop, toolbox,cxpb=0.5, mutpb=0.5,ngen=400, verbose=False)
    best_individual = tools.selBest(result, k=1)[0]
    create_tour(best_individual)
    
plt.figure(figsize=(11, 4))
plots = plt.plot(log.select('min'),'c-', log.select('mean'), 'b-')
plt.legend(plots, ('Minimum fitness', 'Mean fitness'), frameon=True)
plt.ylabel('Fitness'); plt.xlabel('Iterations');


### Extending GA to real data

We are given a set of 14 GPS positions, each coordinate representing a city in Burma (Officially the Republic of the Union of Myanmar). Our objective is to solve the TSP problem over these 14 cities. You need to do bit of independent research for finding the formula to convert the GPS coodinates of two cities (in latitudes and longitudes) to the actual distance between those two cities.

City[i] = {LAT[i], LON[i]}

In [ ]:
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 
        17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 
        96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]


### Q4. Calculate the total distance (in Kilometres) of a tour starting with city[0], going in the order given to city[1], city[2], ... city[13] and coming back to city[0],  based on *Latitude* and *Longitude* of the 14 cities above. [2 Marks] 

In [ ]:
from math import radians, sin, cos, asin, sqrt

def calculateDistance(city1,city2):
    lat1 = city1.real
    long1 = city1.imag
    lat2 = city2.real
    long2 = city2.imag
    lat1, lat2, long1, long2 = map(radians, [lat1, lat2, long1, long2])
    r = 6371000
    latD = lat2 - lat1
    longD = long2 - long1
    p1 = sin(latD/2) ** 2
    p2 = cos(lat1) * cos(lat2)
    p3 = sin(longD/2)**2
    h = p1 + p2 * p3
    d = 2 * r * asin(sqrt(h))
    return d

def total_Circle_distance(tour):
    "The total distance between each pair of consecutive cities in the tour."
    return sum(calculateDistance(tour[i], tour[i-1]) 
               for i in range(len(tour)))

In [ ]:
# Write your code here
Cities = []
for i in range(0,8):
    Cities.append(City(LAT[i],LON[i]))

print(total_Circle_distance(Cities))

plot_tour(Cities)

### Q5. Provide the optimal route you found employing GA, and its length in kilometers.  [2 Marks]

In [ ]:
# Write your code here
def evaluation2(individual):
    '''Evaluates an individual by converting it into 
    a list of cities and passing that list to total_distance'''
    return (total_Circle_distance(create_tour2(individual)),)
2
def create_tour2(individual):
    return [list(Cities)[e] for e in individual]

toolbox = base.Toolbox()
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("indices", numpy.random.permutation, len(Cities))
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)
toolbox.register("population", tools.initRepeat, list, 
                 toolbox.individual)
toolbox.register("evaluate", evaluation2)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

result, log = algorithms.eaSimple(toolbox.population(n=200), toolbox,
                                  cxpb=0.1, mutpb=0.1,
                                  ngen=400, verbose=False,
                                  stats=fit_stats)

best_individual = tools.selBest(result, k=1)[0]
evaluation2(best_individual)

In [ ]:
fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
fit_stats.register('mean', numpy.mean)
fit_stats.register('min', numpy.min)

In [ ]:
plot_tour(create_tour2(best_individual))

### Q6. Describe your fitness function, and the way you encoded the potential solutions. [2 Marks]

### Q7. Provide the configuration of the GA you finally used, namely, mutation probability, crossover probability, population size, type of selection, type of mutation, type of crossover, and number of generations. [2 Marks]

## Solving Tic-Tac-Toe using Genetic Algorithm (GA)
The game of Tic Tac Toe (2D) is played on a 3 by 3 grid.  One player marks her squares with an X, and the other with an O.  The players alternate, placing their marks on an empty cell on the grid with the hope of winning the game.  The winner of the game is the first player to place 3 of their marks in a line (row, column, and diagnoal).  If the entire grid is filled with marks, and there is no winner, the game is declared a draw.  Your task is to create a player to play the 3D version of this game without any knowledge of strategies for playing Tic Tac Toe.  The only rule that is known to this evolving player is that the players alternate their turns, and when it is the evolving player’s turn, it could only place its own mark on an empty spot in the grid. We have already implemented Negamax algorithm to play 3D tic-tac-toe in one of the practicals. In this assignment, we try to solve 3D (3 x 3 x 3) Tic-Tac-Toe game using Genetic Algorithm (GA). 

### 3D (3 x 3 x 3) tic-tac-toe
<img src="3d_tic-tac-toe-3d.jpg" style="width: 200px"/>

### Q8. How did you approach the problem of representing the solution space, and why?  Did you face any specific problem for this – for instance, choosing between different possible representations? How did you overcome any such problem? Give as example how, in your approach, Tic-Tac-Toe (3D version) will be represented, and explain clearly how one can read off the strategy for Tic-Tac-Toe from this “chromosome”. [2 Marks] 

### Q9. What  fitness  function  did you  choose  to  go  with  your  approach,  and  why?   As  before, describe if you faced any specific problem in this context, and how you overcame it. [1 Marks]

### Q10. What parameters for Genetic algorithm did you choose?  For instance what type of crossover did you choose,  with what probability,  and why?  Also outline how you determined that the solution you received is good enough. [2 Marks]

### Q11. Provide your complete code for solving 3D (3 x 3 x 3) Tic-Tac-Toe using Genetic Algorithm. Note that this code should be complete (should not call any methods from the above code for solving TSP). You need to design the layout of the game either in console (command prompt) or Graphical User Interface (GUI) so that human (one of the players) can play against the AI machine (GA). [ 3 Marks]

### On execution of your code:
- it asks the human to pick one of {X, O} as their preferred mark. 
- it lets the human player choose who would start the game.
- After the game is over, it displays the result (who wins/loses or it ended in a draw)

In [ ]:
from deap import algorithms, base, creator, tools
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import random, operator, time, itertools, math
import numpy

toolbox = base.Toolbox()
    
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
    
    ## Create Individuals from possible moves
   # toolbox.register("indices", game.possible_moves)

def GA_TIC_TAC_AI(game):
    toolbox = base.Toolbox()
    
    ## Create Individuals from possible moves
   # toolbox.register("indices", game.possible_moves)
    
    toolbox.register("indices", numpy.random.permutation, game.possible_moves())
    toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, 
                 toolbox.individual)
    
    ## Use fitness function
    hof = tools.HallOfFame(1)
    toolbox.register("evaluate",game.evaluation)
    toolbox.register("mate", tools.cxUniform,indpb=0.1)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
    fit_stats.register('mean', numpy.mean)
    fit_stats.register('max', numpy.max)
    
    pop = toolbox.population(n=150)
    
    pop, log = algorithms.eaSimple(pop, toolbox,
                                  cxpb=0.0, mutpb=0.3,
                                  ngen=250, verbose=False,halloffame=hof, 
                                      stats=fit_stats)
    
    plt.figure(figsize=(11, 4))
    plots = plt.plot(log.select('max'),'c-', log.select('mean'), 'b-')
    plt.legend(plots, ('Maximum fitness', 'Mean fitness'), frameon=True)
    plt.ylabel('Fitness'); plt.xlabel('Iterations');

    return hof[0][0]
    

In [ ]:
## Put your complete code here.
# This is a variant of the Tic Tac Toe recipe given in the easyAI library

from easyAI import TwoPlayersGame, AI_Player, Negamax
from easyAI.Player import Human_Player

class GameController(TwoPlayersGame):
    def __init__(self, players):
        # Define the players
        self.players = players

        self.nplayer = 1 

        # Define the board
        self.board = [0] * 27

    
    # Define possible moves
    def possible_moves(self):
        return [a + 1 for a, b in enumerate(self.board) if b == 0]
    
    # Make a move
    def make_move(self, move):
        self.board[int(move) - 1] = self.nplayer
    
    def makeLoss_Condition(self):
        combos = [[1,2,3], [4,5,6], [7,8,9], [1,4,7], [2,5,8], [3,6,9], [1,5,9], [3,5,7]]
        # 3D Diagonals
        for i in range(1,4):
            # 1-3, diagonal down
            combos.append([i,i+12,i+24])
            # 1,4,7 diagonal right
            combos.append([(i-1)*3+1,(i-1)*3+10,i+20])
            # 7-9 diagonal down
            combos.append([(i+6),(i+6)+6,(i+6)+12])
            # 3,6,9 diagonal left
            combos.append([(i*3)+1,(i-1)*3+8,i+16])
            
        combos.append([1,14,27])
        combos.append([3,14,25])
        combos.append([9,14,19])
        combos.append([7,14,21])
        
        # 3D Directly Z-down
        for i in range(1,10):
            combos.append([i,i+9,i+18])
            
        # For Middle and Bottom Layer
        for i in range (1,3):
            #10-12 down
            for j in range(1,4):
                combos.append([(i*9)+j,(i*9)+j+3,(i*9)+j+6])
            #10,13,16 across
            for j in range(1,4):
                x = (i*9)+(j-1)*3
                combos.append([x+1,x+2,x+3])
                
            #Middle and Bot layer diagonal
            combos.append([(i*9)+1,(i*9)+5,(i*9)+9])
            combos.append([(i*9)+7,(i*9)+5,(i*9)+3])
            
        return combos
            

    # Does the opponent have three in a line?
    def loss_condition(self):
        possible_combinations = self.makeLoss_Condition()

        return any([all([(self.board[i-1] == self.nopponent)
                for i in combination]) for combination in possible_combinations]) 
        
    # Check if the game is over
    def is_over(self):
        if ((self.possible_moves() == [])):
            print("It's a tie")
        
        return (self.possible_moves() == []) or self.loss_condition()
        
    # Show current position
    def show(self):
        print('\n'+'\n'.join([' '
                              .join([['.', 'O', 'X'][self.board[3*j + i]]
                for i in range(3)])
                              for j in range(3)]) + 
              '\n' + '\n' + '\n'.join([' '
                              .join([['.', 'O', 'X'][self.board[3*j + i+9]]
                for i in range(3)]) 
                              for j in range(3)]) +
        '\n' + '\n' + '\n'.join([' '
                              .join([['.', 'O', 'X'][self.board[3*j + i+18]]
                for i in range(3)])
                              for j in range(3)]))

    def make_move1(self, move, board):
        board[int(move) - 1] = self.nplayer
        return board
    
    def make_move2(self, move, board):
        board[int(move) - 1] = self.nopponent 
        return board
    
    def Testloss_condition2(self, board):
        possible_combinations = self.makeLoss_Condition()

        return any([all([(board[i-1] == self.nopponent)
                for i in combination]) for combination in possible_combinations]) 
    
    def Testloss_condition1(self, board):
        possible_combinations = self.makeLoss_Condition()

        return any([all([(board[i-1] == self.nplayer)
                for i in combination]) for combination in possible_combinations]) 
        
    def evaluation(self,individual):
        i = 0
        score = 5
        fakeBoard = self.board.copy()
        while (i < len(individual)-1 ): 
            fakeBoard = self.make_move2(individual[i], fakeBoard)
            if (self.Testloss_condition2(fakeBoard) and i == 0):
                score += 200
                break
            elif (self.Testloss_condition2(fakeBoard)):
                score += 15
            fakeBoard = self.make_move1(individual[i+1],fakeBoard)
            if (self.Testloss_condition1(fakeBoard)):
                score -= 5
            i += 2
        return (score,)

    # Compute the score
    def scoring(self):
        return -100 if self.loss_condition() else 0


if __name__ == "__main__":
    # Define the algorithm
    #algorithm = GA_TIC_TAC_AI
    algo = GA_TIC_TAC_AI
    # Start the game
    letter = ''
    while not (letter == 'X' or letter == 'O'):
        print('Do you want to be X or O?')
        letter = input().upper()
    if (letter == 'O'):
        GameController([Human_Player(),AI_Player(algo)]).play()
    else:
        GameController([AI_Player(algo),Human_Player()]).play()


In [ ]:
# Put your analysis here.

## Your Submission method

Your submission should consist of this jupyter notebook with all your code and explanations inserted in the notebook. The notebook should contain the output of the runs so that it can be read by the assessor without needing to run the output.

You have already used Jupyter Notebook earlier in the unit. In case you need help, you may refer the following tutorial https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook .

Late submissions may attract penalty in accordance with the assessment policy outlined in the unit guide.

Each question specifies a mark. The final mark of the assignment is the sum of all the individual marks, after applying any deductions for late submission.

By submitting this assignment you are acknowledging that this is your own work. Any submissions that break the code of academic honesty will be penalised as per the [academic honesty policy](https://staff.mq.edu.au/work/strategy-planning-and-governance/university-policies-and-procedures/policies/academic-honesty).